In [4]:
import spotipy # don't forget to write: pip install spotipy on gitbash or cmd or whatever
from spotipy.oauth2 import SpotifyClientCredentials
import json
import pandas as pd

In [3]:
# Parse JSON
data = {
    "clientID" : "bc9f3e2fa5994153b808dec97aadb7c8",
    "clientSecret" : "ffb21ad5ba2d486ba533d42aa4091892" 
}
with open('authorization.json', 'w') as outfile:
    json.dump(data, outfile)

In [7]:
# Reading JSON file as a dataframe
# clients = pd.read_json("authorization.json")

# Reading JSON file
# Python Parsing JSON 
with open('authorization.json') as json_file:
    data = json.load(json_file)
data

{'clientID': 'bc9f3e2fa5994153b808dec97aadb7c8',
 'clientSecret': 'ffb21ad5ba2d486ba533d42aa4091892'}